In [284]:
import pandas as pd
import json
import numpy as np
from google.cloud import bigquery

In [240]:
#conversion du fichier json en json line delimiter pour utiliser pd.read_json avec une chunk size. (Nombre de ligne processée en une fois)
#!cat ../raw_data/export_23-5_minified.json | jq -c '.[]' > ../raw_data/export_converted_2.json

In [241]:
#Nombre de lignes totales dans le JSON : 1320229

In [242]:
#Localisation du projet name et table_id sur gcp
project="marbotic"
table_id = "marbotic_dataset.user_properties"

In [243]:
#intégration des credentials 
from google.oauth2 import service_account

key_path = "/Users/antonin/code/AntoninAnq/gcp/marbotic-7d02fac30bd8.json"

credentials = service_account.Credentials.from_service_account_file(
    key_path, scopes=["https://www.googleapis.com/auth/cloud-platform"],
)

In [290]:
# Observation de la colonne Games, on a des listes de jeu. Sur le chunk de 100K on a 158 combinaisons différentes
tmp = pd.read_csv('../raw_data/user_properties.csv')
tmp
#pd.DataFrame(tmp.Games.to_list())
#.event_id.value_counts()
#.Games.value_counts().head(20)

/var/folders/c7/kdqvz76x679f28lhd235r_v40000gn/T/ipykernel_74454/3989991794.py:2: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  tmp = pd.read_csv('../raw_data/user_properties.csv')


,Newsletter,Products,Pieces,Type,Country,Games,Language,ProfileType2,ReceiveNewsletter,LoggedIn,event_id
0,False,"['Smart Letters', 'Smart Numbers']",English,Parent,US,['Marbotic'],NaN,NaN,NaN,NaN,1110
1,False,"['Smart Letters', 'Smart Numbers']","['Uppercase Letters', 'Numbers']",Parent,US,['Marbotic'],English,NaN,NaN,NaN,1111
2,True,"['Smart Letters', 'Smart Numbers']","['Uppercase Letters', 'Numbers']",Parent,CA,"['Marbotic Numbers', 'Marbotic Letters', 'Alph...",English,NaN,NaN,NaN,1430
3,True,"['Smart Letters', 'Smart Numbers']","['Uppercase Letters', 'Numbers']",Parent,CA,"['Marbotic Numbers', 'Marbotic Letters', 'Alph...",English,NaN,NaN,NaN,1431
4,True,"['Smart Letters', 'Smart Numbers']","['Uppercase Letters', 'Numbers']",Parent,CA,"['Marbotic Numbers', 'Marbotic Letters', 'Alph...",English,NaN,NaN,NaN,1432
...,...,...,...,...,...,...,...,...,...,...,...
99995,True,['Smart Letters Lowercase'],['Lowercase Letters'],Parent,US,['Marbotic'],English,NaN,NaN,NaN,544
99996,True,['Smart Letters Lowercase'],['Lowercase Letters'],Parent,US,['Marbotic'],English,NaN,NaN,NaN,545
99997,True,['Smart Letters Lowercase'],['Lowercase Letters'],Parent,US,['Marbotic'],English,NaN,NaN,NaN,546
99998,True,['Smart Letters Lowercase'],['Lowercase Letters'],Parent,US,['Marbotic'],English,NaN,NaN,NaN,547


In [288]:
job_config = bigquery.LoadJobConfig(
    schema = [
    bigquery.SchemaField("Newsletter", "STRING"),
    #bigquery.SchemaField("Products", "STRING"),
    #bigquery.SchemaField("Pieces", "STRING"),
    bigquery.SchemaField("Type", "STRING"),
    bigquery.SchemaField("Country", "STRING"),
    bigquery.SchemaField("Language", "STRING"),
    bigquery.SchemaField("ProfileType2", "STRING"),
    bigquery.SchemaField("ReceiveNewsletter", "STRING"),
    bigquery.SchemaField("LoggedIn", "STRING"),
    bigquery.SchemaField("event_id", "INT64"),
    bigquery.SchemaField(
        "Games",
        "RECORD",
        mode="REPEATED",
        fields=[
            bigquery.SchemaField("Marbotic", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("Smart Shapes", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("Alphamonster", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("10 Fingers", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("Up To 100", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("Lil Reader", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("More Or Less", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("Vocabubble", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("Marbotic Numbers", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("Bla Bla Box", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("Marbotic Letters", "STRING", mode="NULLABLE"),
        ],
    ),
])

In [328]:
CHUNKSIZE = 100000

file_path = '../raw_data/export_converted_2.json'
df = pd.read_json(file_path, lines=True, chunksize=CHUNKSIZE) #Chunk size pour avoir 14 chunk

client = bigquery.Client(project,credentials)

for i,c in enumerate(df):
    index_init = np.linspace(1,len(c),len(c),dtype='int32')
    ep=pd.DataFrame()
    c.reset_index(inplace=True)
    ep=pd.DataFrame(c["user_properties"].to_list()).copy()
    ep.drop(['Products','Pieces','Games'],axis=1,inplace=True)
    #ep[['Products','Pieces','Games']]=ep[['Products','Pieces','Games']].astype(str)
    ep['event_id']=c["event_id"].copy()
    ep['id'] = (i*CHUNKSIZE) + index_init
    #ep.to_csv('../raw_data/event_properties.csv',index=False)
    job = client.load_table_from_dataframe(ep, table_id)  # Make an API request. ,job_config=job_config
    job.result()  # Wait for the job to complete.
    table = client.get_table(table_id)  # Make an API request.
    print(
        "Loaded {} rows and {} columns to {}".format(
            table.num_rows, len(table.schema), table_id
        )
    )

Loaded 100000 rows and 9 columns to marbotic_dataset.user_properties
Loaded 200000 rows and 9 columns to marbotic_dataset.user_properties
Loaded 300000 rows and 9 columns to marbotic_dataset.user_properties
Loaded 400000 rows and 9 columns to marbotic_dataset.user_properties
Loaded 500000 rows and 9 columns to marbotic_dataset.user_properties
Loaded 600000 rows and 9 columns to marbotic_dataset.user_properties
Loaded 700000 rows and 9 columns to marbotic_dataset.user_properties
Loaded 800000 rows and 9 columns to marbotic_dataset.user_properties
Loaded 900000 rows and 9 columns to marbotic_dataset.user_properties
Loaded 1000000 rows and 9 columns to marbotic_dataset.user_properties
Loaded 1100000 rows and 9 columns to marbotic_dataset.user_properties
Loaded 1200000 rows and 9 columns to marbotic_dataset.user_properties
Loaded 1300000 rows and 9 columns to marbotic_dataset.user_properties
Loaded 1320229 rows and 9 columns to marbotic_dataset.user_properties
